In [ ]:
!pip install transformers pandas


In [ ]:
import os

tar_file = 'liputan6_data.tar.gz'
!tar -xzvf /content/{tar_file} -C /content/


tar (child): /content/liputan6_data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
!ls /content/liputan6_data/canonical/dev/


ls: cannot access '/content/liputan6_data/canonical/dev/': No such file or directory


In [ ]:
import pandas as pd
import json

In [ ]:
import os

# Nama file .tar.gz Anda
tar_file = 'liputan6_data.tar.gz'

# Ekstrak file ke folder saat ini
!tar -xzvf /content/{tar_file} -C /content/


tar (child): /content/liputan6_data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
!ls /content/


sample_data


In [ ]:
!ls /content/liputan6_data/canonical/dev/


ls: cannot access '/content/liputan6_data/canonical/dev/': No such file or directory


In [ ]:
for key, value in data.items():
    print(f"Key: {key}, Type: {type(value)}, Length: {len(value) if hasattr(value, '__len__') else 'N/A'}")


NameError: name 'data' is not defined

In [ ]:
# Cari panjang minimum dari elemen-elemen yang berbentuk list
min_length = min(len(data[key]) for key in data if isinstance(data[key], list))
print(f"Panjang minimum: {min_length}")


NameError: name 'data' is not defined

In [ ]:
dataset["id"] = [data["id"]] * min_length


In [ ]:
# Potong elemen list ke panjang minimum
data_cleaned = {
    key: (value[:min_length] if isinstance(value, list) else value)
    for key, value in data.items()
}

# Periksa hasil setelah pemotongan
for key, value in data_cleaned.items():
    print(f"Key: {key}, Length: {len(value) if isinstance(value, list) else 'N/A'}")


Key: id, Length: N/A
Key: url, Length: N/A
Key: clean_article, Length: 2
Key: clean_summary, Length: 2
Key: extractive_summary, Length: 2


In [ ]:
import pandas as pd

# Buat DataFrame dari key yang relevan
dataset = pd.DataFrame({
    "url": data_cleaned["url"][:min_length],
    "clean_article": data_cleaned["clean_article"][:min_length],
    "clean_summary": data_cleaned["clean_summary"][:min_length],
    "extractive_summary": data_cleaned["extractive_summary"][:min_length],
})

# Tampilkan beberapa baris pertama
print(dataset.head())


  url                                      clean_article  \
0  ht  [Liputan6, ., com, ,, Ambon, :, Solusi, buat, ...   
1  ht  [Sebaiknya, elite, politik, di, Jakarta, seger...   

                                       clean_summary  extractive_summary  
0  [Khusus, penanganan, masalah, di, Maluku, ,, s...                   1  
1  [Konflik, di, pusat, malah, memperparah, konfl...                   2  


In [ ]:
dataset.to_csv('/content/cleaned_dataset.csv', index=False)
print("Dataset cleaned and saved as CSV.")


Dataset cleaned and saved as CSV.


In [ ]:
dataset["id"] = [data["id"]] * min_length


In [ ]:
!pip install transformers


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Muat tokenizer dan model T5
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [ ]:
# Ubah semua elemen dalam kolom menjadi string
dataset['clean_article'] = dataset['clean_article'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))


In [ ]:
# Ubah semua elemen dalam kolom menjadi string
dataset['clean_summary'] = dataset['clean_summary'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))


In [ ]:
# Ubah semua elemen dalam kolom menjadi string
dataset['extractive_summary'] = dataset['extractive_summary'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))


In [ ]:
# Contoh teks dari clean_article
input_text = dataset['clean_article'][0]

# Tambahkan prefix 'summarize:'
input_text = "summarize: " + input_text

# Tokenisasi teks
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# Lakukan inferensi dengan model
outputs = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode hasil summarization
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Ringkasan:\n", summary)


Ringkasan:
 yang mencuri ikan di kawasan perairan Indonesia tak juga berakhir. nelayan asing yang mencuri ikan di kawasan perairan.


In [ ]:
# Konversi semua elemen ke string jika ada kolom lain yang berisi list
dataset['clean_article'] = dataset['clean_article'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

# Loop summarization
summaries = []
for article in dataset['clean_article']:
    input_text = "summarize: " + article
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summaries.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Tambahkan kolom hasil ringkasan
dataset['generated_summary'] = summaries


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=291dec721de2f01cdb7098bf4c22607302b059dd6170760b2e2931d02e6e7039
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

# Inisialisasi ROUGE Scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Hitung ROUGE untuk setiap ringkasan
rouge_scores = []
for ref, gen in zip(dataset['clean_summary'], dataset['generated_summary']):
    # Pastikan ringkasan adalah string
    ref = ' '.join(ref) if isinstance(ref, list) else str(ref)
    gen = ' '.join(gen) if isinstance(gen, list) else str(gen)

    scores = scorer.score(ref, gen)
    rouge_scores.append(scores)

# Tampilkan beberapa skor sebagai contoh
print("Contoh Skor ROUGE:")
for i, score in enumerate(rouge_scores[:5]):
    print(f"Sample {i+1}: {score}")


Contoh Skor ROUGE:
Sample 1: {'rouge1': Score(precision=0.07692307692307693, recall=0.058823529411764705, fmeasure=0.06666666666666667), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.07692307692307693, recall=0.058823529411764705, fmeasure=0.06666666666666667)}
Sample 2: {'rouge1': Score(precision=0.08333333333333333, recall=0.125, fmeasure=0.1), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.08333333333333333, recall=0.125, fmeasure=0.1)}


In [ ]:
import numpy as np

# Ekstrak nilai rata-rata ROUGE
rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

print(f"ROUGE-1 Average: {rouge1:.4f}")
print(f"ROUGE-2 Average: {rouge2:.4f}")
print(f"ROUGE-L Average: {rougeL:.4f}")


ROUGE-1 Average: 0.0833
ROUGE-2 Average: 0.0000
ROUGE-L Average: 0.0833


In [ ]:
# Format dataset untuk fine-tuning
training_data = pd.DataFrame({
    "input_text": "summarize: " + dataset['clean_article'],  # Tambahkan prefix "summarize:"
    "target_text": dataset['clean_summary']
})

# Simpan dataset
training_data.to_csv('/content/training_data.csv', index=False)
print("Training data saved to '/content/training_data.csv'.")


Training data saved to '/content/training_data.csv'.


In [ ]:
!pip install transformers datasets


In [ ]:
import pandas as pd

# Baca file CSV untuk memeriksa isinya
training_data = pd.read_csv('/content/training_data.csv')
print(training_data.head())


                                          input_text  \
0  summarize: Liputan6 . com , Ambon : Solusi bua...   
1  summarize: Sebaiknya elite politik di Jakarta ...   

                                         target_text  
0  Khusus penanganan masalah di Maluku , sebaikny...  
1  Konflik di pusat malah memperparah konflik di ...  


In [ ]:
from datasets import load_dataset

# Muat dataset pelatihan
dataset = load_dataset(
    'csv',
    data_files={'train': '/content/training_data.csv'}
)

# Periksa dataset
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 2
    })
})


In [ ]:
from transformers import T5Tokenizer

# Muat tokenizer T5
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Fungsi tokenisasi
def preprocess_function(examples):
    inputs = [ex for ex in examples["input_text"]]
    targets = [ex for ex in examples["target_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenisasi dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/t5-fine-tuned",
    evaluation_strategy="no",  # Nonaktifkan evaluasi jika hanya untuk pelatihan
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir="/content/logs",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import os

# Nonaktifkan W&B
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/t5-fine-tuned",
    evaluation_strategy="no",  # Nonaktifkan evaluasi jika hanya untuk pelatihan
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir="/content/logs",
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer

# Muat model T5
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    tokenizer=tokenizer,
)

# Mulai pelatihan
trainer.train()


<ipython-input-88-b9fa8319a86b>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=3, training_loss=10.882455190022787, metrics={'train_runtime': 21.4266, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.14, 'total_flos': 812050808832.0, 'train_loss': 10.882455190022787, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/fine_tuned_t5")
tokenizer.save_pretrained("/content/fine_tuned_t5")
print("Model fine-tuned saved successfully.")


Model fine-tuned saved successfully.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Muat model dan tokenizer dari lokasi penyimpanan
model = T5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5")
tokenizer = T5Tokenizer.from_pretrained("/content/fine_tuned_t5")

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [ ]:
[
    {"input_text": "Artikel lengkap", "target_text": "Ringkasan"},
    {"input_text": "Artikel lain", "target_text": "Ringkasan lain"}
]


[{'input_text': 'Artikel lengkap', 'target_text': 'Ringkasan'},
 {'input_text': 'Artikel lain', 'target_text': 'Ringkasan lain'}]

In [ ]:
for key, value in data.items():
    print(f"Key: {key}, Type: {type(value)}, Length: {len(value) if hasattr(value, '__len__') else 'N/A'}")


Key: id, Type: <class 'int'>, Length: N/A
Key: url, Type: <class 'str'>, Length: 94
Key: clean_article, Type: <class 'list'>, Length: 11
Key: clean_summary, Type: <class 'list'>, Length: 2
Key: extractive_summary, Type: <class 'list'>, Length: 2


In [ ]:
# Potong elemen list ke panjang minimum
data_cleaned = {
    key: (value[:min_length] if isinstance(value, list) else value)
    for key, value in data.items()
}

# Periksa hasil setelah pemotongan
for key, value in data_cleaned.items():
    print(f"Key: {key}, Length: {len(value) if isinstance(value, list) else 'N/A'}")


Key: id, Length: N/A
Key: url, Length: N/A
Key: clean_article, Length: 2
Key: clean_summary, Length: 2
Key: extractive_summary, Length: 2


In [ ]:
{
    "input_text": ["Artikel lengkap", "Artikel lain"],
    "target_text": ["Ringkasan", "Ringkasan lain"]
}


{'input_text': ['Artikel lengkap', 'Artikel lain'],
 'target_text': ['Ringkasan', 'Ringkasan lain']}

In [ ]:
import pandas as pd

# Ubah dictionary menjadi DataFrame
data = {
    'input_text': ['Artikel lengkap', 'Artikel lain'],
    'target_text': ['Ringkasan', 'Ringkasan lain']
}

test_data = pd.DataFrame(data)

# Periksa hasil DataFrame
print(test_data.head())


        input_text     target_text
0  Artikel lengkap       Ringkasan
1     Artikel lain  Ringkasan lain


In [ ]:
import json

# Muat file JSON
with open('/content/liputan6_data/canonical/test/15900.json', 'r') as f:
    data = json.load(f)

# Ubah menjadi DataFrame
test_data = pd.DataFrame(data)

# Periksa hasil DataFrame
print(test_data.head())


ValueError: All arrays must be of the same length

In [ ]:
!ls /content/liputan6_data/canonical/test/


In [ ]:
with open('/content/liputan6_data/canonical/test/13303.json', 'r') as f:
    data = json.load(f)


In [ ]:
# Potong elemen list ke panjang minimum
data_cleaned = {
    key: (value[:min_length] if isinstance(value, list) else value)
    for key, value in data.items()
}

# Periksa hasil setelah pemotongan
for key, value in data_cleaned.items():
    print(f"Key: {key}, Length: {len(value) if isinstance(value, list) else 'N/A'}")


In [ ]:
import os
import pandas as pd
import json

# Folder tempat file JSON berada
folder_path = '/content/liputan6_data/canonical/test/'

# List semua file JSON di folder
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Gabungkan semua file JSON menjadi satu DataFrame
all_data = []

for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    with open(file_path, 'r') as f:
        data = json.load(f)
        all_data.append(pd.DataFrame(data))

# Gabungkan semua DataFrame
final_test_data = pd.concat(all_data, ignore_index=True)

# Periksa hasil
print(final_test_data.head())


In [ ]:
with open('/content/liputan6_data/canonical/test/13303.json', 'r') as f:
    data = json.load(f)

# Periksa panjang array untuk setiap key
for key, value in data.items():
    print(f"Key: {key}, Length: {len(value) if isinstance(value, list) else 'N/A'}")


In [ ]:
# Cari panjang minimum
min_length = min(len(value) for value in data.values() if isinstance(value, list))

# Potong semua array ke panjang minimum
data_trimmed = {key: value[:min_length] if isinstance(value, list) else value for key, value in data.items()}

# Ubah menjadi DataFrame
import pandas as pd
test_data = pd.DataFrame(data_trimmed)

# Periksa DataFrame
print(test_data.head())


In [ ]:
import os
import pandas as pd
import json

# Folder tempat file JSON berada
folder_path = '/content/liputan6_data/canonical/test/'

# List semua file JSON
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Gabungkan semua file JSON menjadi satu DataFrame
all_data = []

for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    with open(file_path, 'r') as f:
        data = json.load(f)

        # Selaraskan panjang array di setiap file
        min_length = min(len(value) for value in data.values() if isinstance(value, list))
        data_trimmed = {key: value[:min_length] if isinstance(value, list) else value for key, value in data.items()}

        # Tambahkan ke daftar DataFrame
        all_data.append(pd.DataFrame(data_trimmed))

# Gabungkan semua DataFrame
final_test_data = pd.concat(all_data, ignore_index=True)

# Periksa hasil
print(final_test_data.head())


In [ ]:
final_test_data.to_csv('/content/final_test_data.csv', index=False)
print("Data saved to /content/final_test_data.csv")


In [ ]:
import pandas as pd

# Muat data dev
dev_data = pd.read_csv('/content/training_data.csv')  # Ganti dengan nama file dev Anda

# Periksa data
print(dev_data.head())


In [ ]:
# Muat data test
test_data = pd.read_csv('/content/final_test_data.csv')

# Periksa data
print(test_data.head())


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model dan tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/fine_tuned_t5")
tokenizer = T5Tokenizer.from_pretrained("/content/fine_tuned_t5")

print("Model and tokenizer loaded successfully!")


In [ ]:
print(dev_data.columns)  # Untuk data dev
print(test_data.columns)  # Untuk data test


In [ ]:
# Gunakan nama kolom yang ditemukan dalam dataset
dev_data['generated_summary'] = dev_data['input_text'].apply(generate_summary)


NameError: name 'dev_data' is not defined

In [ ]:
# Inferensi pada data test
test_data['clean_summary'] = test_data['clean_article'].apply(generate_summary)